In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!nvidia-smi

Wed Mar  6 04:11:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
root_dir = '/content/gdrive/MyDrive/Dataset/Drone'

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.9/78.2 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display,Image

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/gdrive/MyDrive/Dataset/Drone/data.yaml epochs=3 plots=True

100% 21.5M/21.5M [00:00<00:00, 131MB/s] 
Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/gdrive/MyDrive/Dataset/Drone/data.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, 

In [ ]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.25 source="/content/gdrive/MyDrive/Dataset/Drone/istockphoto-1305087512-640_adpp_is" save_txt=True

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Define path to video file
source = '/content/gdrive/MyDrive/Dataset/Drone/istockphoto-1305087512-640_adpp_is.mp4'

# Run inference on the source
results = model(source, save=True)

In [ ]:
import zipfile
import os

def zip_folder(folder_path, output_path):
    """
    Compresses the folder at the given folder_path into a zip file at the given output_path.
    """
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zip_file.write(file_path, os.path.relpath(file_path, folder_path))

# Example usage
zip_folder('/content/runs/detect/predict', '/content/runs/detect/predict.zip')

In [ ]:
import cv2
import math
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

model = YOLO("/content/runs/detect/train/weights/best.pt")
cap = cv2.VideoCapture("/content/gdrive/MyDrive/Dataset/Drone/istockphoto-1305087512-640_adpp_is.mp4")

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

out = cv2.VideoWriter('visioneye-distance-calculation.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (w, h))

center_point = (0, h)
pixel_per_meter = 10

txt_color, txt_background, bbox_clr = ((0, 0, 0), (255, 255, 255), (255, 0, 255))

while True:
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    annotator = Annotator(im0, line_width=2)

    results = model.track(im0, persist=True)
    boxes = results[0].boxes.xyxy.cpu()

    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for box, track_id in zip(boxes, track_ids):
            annotator.box_label(box, label=str(track_id), color=bbox_clr)
            annotator.visioneye(box, center_point)

            x1, y1 = int((box[0] + box[2]) // 2), int((box[1] + box[3]) // 2)    # Bounding box centroid

            distance = (math.sqrt((x1 - center_point[0]) ** 2 + (y1 - center_point[1]) ** 2))/pixel_per_meter

            text_size, _ = cv2.getTextSize(f"Distance: {distance:.2f} m", cv2.FONT_HERSHEY_SIMPLEX,0.5, 2)
            cv2.rectangle(im0, (x1, y1 - text_size[1] - 10),(x1 + text_size[0] + 10, y1), txt_background, -1)
            cv2.putText(im0, f"Distance: {distance:.2f} m",(x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,txt_color, 2)

    out.write(im0)
    # cv2.imshow("visioneye-distance-calculation", im0)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out.release()
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 drone, 11.7ms
Speed: 1.6ms preprocess, 11.7ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 22.4ms
Speed: 1.2ms preprocess, 22.4ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 14.4ms
Speed: 1.7ms preprocess, 14.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 13.3ms
Speed: 1.5ms preprocess, 13.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 10.8ms
Speed: 1.1ms preprocess, 10.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 10.6ms
Speed: 2.5ms preprocess, 10.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 11.8ms
Speed: 1.3ms preprocess, 11.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 10.6ms
Speed: 2.1ms preprocess, 10.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
